<a href="https://colab.research.google.com/github/mandie021/big-data-challenge/blob/main/level-1/level_1_outdoor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:13 https://developer.

In [2]:
#import postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-03-02 16:02:05--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.76MB/s    in 0.5s    

2022-03-02 16:02:06 (1.76 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
#create session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Outdoors_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
outdoor_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Outdoors_v1_00.tsv.gz"), sep="\t", header=True)
outdoor_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18446823|R35T75OLUGHL5C|B000NV6H94|     110804376|Stearns Youth Boa...|        Outdoors|          4|            0|          0|   N|                Y|          Four Stars|          GOOD VALUE| 2015-08-31|
|         US|   13724367|R2BV735O46BN33|B000IN0W3Y|     624096774|Primal Wear Men's...|        Outdoors|          5|    

In [5]:
#Remove duplicates and na values
outdoor_df = outdoor_df.dropna()
outdoor_df = outdoor_df.dropDuplicates()

In [6]:
outdoor_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [7]:
# Count the number of records (rows) in the dataset
outdoor_df.count()

2302174

In [22]:
# Transform the dataset to fit the tables in the schema file
#imports and review id table
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import to_date

review_id_table = outdoor_df.select(["review_id",  "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1001QVXPSXM5P|   44206795|B006ZUQN28|     759580599| 2014-04-23|
|R100GQY0OPWJU5|   20053688|B00E92OQTG|     297577970| 2015-07-04|
|R100GRJK9XS6LR|    2227673|B00FXS79CG|     537020779| 2014-06-14|
|R100PCZUAU2CGX|   33861887|B004N62HMQ|     483002054| 2013-12-11|
| R1019I5T6LHSS|   46831274|B00IJIWQQQ|      36844749| 2015-01-09|
|R101INERILHDIC|   16215158|B006JMKL66|     551037714| 2014-07-27|
|R102F4ZCBPKI3C|   46775668|B006JJMGYY|     412215106| 2015-08-11|
|R104IGYZLG628Y|   11183179|B003BMDI9E|     469532855| 2015-03-14|
|R104PBWKPR9CSK|   31468397|B002FVYTLI|     758912772| 2013-07-26|
|R10562YCEPQ7PE|   21620567|B00DGBO7U4|     220099064| 2015-03-23|
|R1058AGPJCRJMY|   13456458|B0012LFXSA|     379546348| 2014-05-27|
|R105TYOK8OCVES|   11725589|B000CSKBBS|     125505203| 2013-04

In [31]:
#make columns int
review_id_table = review_id_table.withColumn("customer_id", review_id_table["customer_id"].cast(IntegerType()))
review_id_table = review_id_table.withColumn("product_parent", review_id_table["product_parent"].cast(IntegerType()))

In [32]:
review_id_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [9]:
# Transform the dataset to fit the tables in the schema file
#imports and products table
from pyspark import SparkFiles
products = outdoor_df.select(["product_id", "product_title"])
products = products.drop_duplicates(["product_id", "product_title"])
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00IFHFJXI|Ivation Portable ...|
|B00WG0J0D0|JanSport Superbre...|
|B00V15AUN0|Nickelodeon Paw P...|
|B00FUWSTI8|Bago Lightweight ...|
|B003FV94NA|Michelin Lithion ...|
|B00WIK04HO|Ultra Bright Camp...|
|B00J2HSCM0|High Sierra Tank ...|
|B009I6NSR4|Black Veil Brides...|
|B001GSHSLE|Stansport 191 App...|
|B00L2IO9M4|Columbia Sportswe...|
|B00KY7IM7W|Nalgene 32 Oz Wid...|
|B00TV5JCTK|Rollerblade ABEC ...|
|B00B9D071Y|BUFF UV Multifunc...|
|B00F9IGIKO|Condor Tactical F...|
|B004X55L9I|Hydro Flask Insul...|
|B00LORROIY|Scuba Choice Divi...|
|B00AATRU8G|Kelty Redwing 44 ...|
|B00HMCYWEO|Dakine Explorer L...|
|B004DK1CM8|Hot Headz 12V Hea...|
|B00T4W6SSS|Fits Sock Light H...|
+----------+--------------------+
only showing top 20 rows



In [33]:
products.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [10]:
# Transform the dataset to fit the tables in the schema file
#imports and customers table
customers = outdoor_df.groupby(["customer_id"]).count()
customers = customers.withColumnRenamed("count", "customer_count")
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   15152396|             1|
|   27765587|             1|
|   48428870|            22|
|   14130300|             1|
|   40373996|             3|
|   34027045|             1|
|     129880|             1|
|   17180966|             1|
|   43190041|             1|
|   12341684|             1|
|     119655|             1|
|   28632693|             3|
|   38079447|             1|
|   46752395|             1|
|   32875963|             1|
|   42043208|             1|
|   44378093|             1|
|   41816283|             1|
|   39467281|             5|
|   23142324|             8|
+-----------+--------------+
only showing top 20 rows



In [36]:
#convert items to int
customers = customers.withColumn("customer_id", customers["customer_id"].cast(IntegerType()))
customers = customers.withColumn("customer_count", customers["customer_count"].cast(IntegerType()))

In [37]:
customers.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [11]:
# Transform the dataset to fit the tables in the schema file
#imports and vine table
vine_table = outdoor_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1001QVXPSXM5P|          4|            2|          3|   N|
|R100GQY0OPWJU5|          5|            0|          0|   N|
|R100GRJK9XS6LR|          4|            0|          0|   N|
|R100PCZUAU2CGX|          5|            1|          1|   N|
| R1019I5T6LHSS|          3|            0|          0|   N|
|R101INERILHDIC|          4|            0|          0|   N|
|R102F4ZCBPKI3C|          5|            0|          0|   N|
|R104IGYZLG628Y|          4|            1|          1|   N|
|R104PBWKPR9CSK|          4|            0|          0|   N|
|R10562YCEPQ7PE|          5|            0|          0|   N|
|R1058AGPJCRJMY|          1|            1|          1|   N|
|R105TYOK8OCVES|          1|            0|          0|   N|
|R105YO1H8F9XU8|          5|            1|          1|   N|
|R106FSAW58OUZW|          4|            

In [39]:
#convert items to int
vine_table = vine_table.withColumn("star_rating", vine_table["star_rating"].cast(IntegerType()))
vine_table = vine_table.withColumn("helpful_votes", vine_table["helpful_votes"].cast(IntegerType()))
vine_table = vine_table.withColumn("total_votes", vine_table["total_votes"].cast(IntegerType()))

In [40]:
vine_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [13]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://bigdatahwdb.crkggtvwjikn.us-east-1.rds.amazonaws.com:5432/hw_lvl1_db"
config = {"user":"root", 
          "password": "****", 
          "driver":"org.postgresql.Driver"}


In [41]:
# Write DataFrame to review_id table in RDS
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [42]:
# Write DataFrame to products table in RDS
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [43]:
# Write DataFrame to customers table in RDS
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [44]:
# Write DataFrame to vine_table table in RDS
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)